# Getting timestamps of Swedish Librivox

> "With wav2vec2 and HuggingFace transformers"

- tok: false
- branch: master
- comments: true
- categories: [long audio, wav2vec2, huggingface, timestamps, librivox, swedish]


In [1]:
DONE = """
ennyckfullkvinna2_12.wav
djaeknarne_10.wav
ennyckfullkvinna2_05.wav
ennyckfullkvinna2_04.wav
ennyckfullkvinna_09.wav
jaochnej_04.wav
goteborgsflickor_07.wav
ennyckfullkvinna_01.wav
"""

In [2]:
DATA = """
#avskedssang	https://www.archive.org/download/multilingual_poetry_018_1112_lv/swedish_avskedssang_andersson_e.mp3	http://runeberg.org/daefter/12.html	Elin	https://librivox.org/reader/6686	f
efter_torgdagen	https://www.archive.org/download/mshortworks_001_1202_librivox/msw001_19_eftertorsdagen_benedictsson_at.mp3	https://litteraturbanken.se/f%C3%B6rfattare/BenedictssonV/titlar/Ber%C3%A4ttelserOchUtkast/sida/126/etext	owly	https://librivox.org/reader/2857	f
en_saga_om_vreden	https://ia800801.us.archive.org/0/items/mshortworks_001_1202_librivox/msw001_12_en_saga_om_vreden_runeberg_jb.mp3	http://web.archive.org/web/20190814032041/http://freetexthost.com:80/bcp31m60i4	Johan Borg	https://librivox.org/reader/5958	m
elin_i_hagen	https://www.archive.org/download/multilingual_poetry_014_1002/swedish_elinihagen_froding_ear.mp3	http://runeberg.org/dragharm/elinhage.html	Elina Riuttanen	https://librivox.org/reader/4498	f
djaeknarne_01	https://www.archive.org/download/vandrande_draeknarne_0907_librivox/djaeknarne_01_rydberg.mp3	https://www.gutenberg.org/cache/epub/9827/pg9827.txt	Lars Rolander (1942-2016)	https://librivox.org/reader/2234	m
djaeknarne_02	https://www.archive.org/download/vandrande_draeknarne_0907_librivox/djaeknarne_02_rydberg.mp3	https://www.gutenberg.org/cache/epub/9827/pg9827.txt	Lars Rolander (1942-2016)	https://librivox.org/reader/2234	m
djaeknarne_03	https://www.archive.org/download/vandrande_draeknarne_0907_librivox/djaeknarne_03_rydberg.mp3	https://www.gutenberg.org/cache/epub/9827/pg9827.txt	Lars Rolander (1942-2016)	https://librivox.org/reader/2234	m
djaeknarne_04	https://www.archive.org/download/vandrande_draeknarne_0907_librivox/djaeknarne_04_rydberg.mp3	https://www.gutenberg.org/cache/epub/9827/pg9827.txt	Lars Rolander (1942-2016)	https://librivox.org/reader/2234	m
djaeknarne_05	https://www.archive.org/download/vandrande_draeknarne_0907_librivox/djaeknarne_05_rydberg.mp3	https://www.gutenberg.org/cache/epub/9827/pg9827.txt	Lars Rolander (1942-2016)	https://librivox.org/reader/2234	m
djaeknarne_06	https://www.archive.org/download/vandrande_draeknarne_0907_librivox/djaeknarne_06_rydberg.mp3	https://www.gutenberg.org/cache/epub/9827/pg9827.txt	Lars Rolander (1942-2016)	https://librivox.org/reader/2234	m
djaeknarne_07	https://www.archive.org/download/vandrande_draeknarne_0907_librivox/djaeknarne_07_rydberg.mp3	https://www.gutenberg.org/cache/epub/9827/pg9827.txt	Lars Rolander (1942-2016)	https://librivox.org/reader/2234	m
djaeknarne_08	https://www.archive.org/download/vandrande_draeknarne_0907_librivox/djaeknarne_08_rydberg.mp3	https://www.gutenberg.org/cache/epub/9827/pg9827.txt	Lars Rolander (1942-2016)	https://librivox.org/reader/2234	m
djaeknarne_09	https://www.archive.org/download/vandrande_draeknarne_0907_librivox/djaeknarne_09_rydberg.mp3	https://www.gutenberg.org/cache/epub/9827/pg9827.txt	Lars Rolander (1942-2016)	https://librivox.org/reader/2234	m
djaeknarne_10	https://www.archive.org/download/vandrande_draeknarne_0907_librivox/djaeknarne_10_rydberg.mp3	https://www.gutenberg.org/cache/epub/9827/pg9827.txt	Lars Rolander (1942-2016)	https://librivox.org/reader/2234	m
djaeknarne_11	https://www.archive.org/download/vandrande_draeknarne_0907_librivox/djaeknarne_11_rydberg.mp3	https://www.gutenberg.org/cache/epub/9827/pg9827.txt	Lars Rolander (1942-2016)	https://librivox.org/reader/2234	m
djaeknarne_12	https://www.archive.org/download/vandrande_draeknarne_0907_librivox/djaeknarne_12_rydberg.mp3	https://www.gutenberg.org/cache/epub/9827/pg9827.txt	Lars Rolander (1942-2016)	https://librivox.org/reader/2234	m
elddonet	https://ia903002.us.archive.org/29/items/skandinaviskaaventyr_1907_librivox/skandinaviskaeventyr_01_various_64kb.mp3	http://runeberg.org/hcasob/1/0003.html	Rikard Peterson	https://librivox.org/reader/10462	m
for_morgonbris	https://www.archive.org/download/multilingual_poetry_collection_007_librivox/swedish_for_morgonbris_tavaststjerna_ck.mp3	https://web.archive.org/web/20100413161537/http://www.omnibus.se:80/cgi-bin/ebokinfo.pl?eB=395-1	Cecilia	https://librivox.org/reader/919	f
goteborgsflickor_01	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_01_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_02	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_02_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_03	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_03_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_04	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_04_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_05	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_05_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_06	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_06_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_07	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_07_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_08	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_08_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_09	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_09_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_10	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_10_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_11	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_11_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_12	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_12_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_13	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_13_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_14	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_14_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_15	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_15_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_16	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_16_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_17	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_17_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_18	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_18_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
goteborgsflickor_19	https://www.archive.org/download/goteborgsflickor_1005_librivox/goteborgsflickor_19_stroemberg.mp3	https://www.gutenberg.org/files/18043/18043-0.txt	  Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
jaochnej_01	https://www.archive.org/download/ja_och_nej_0903_librivox/jaochnej_01_nyblom.mp3	http://runeberg.org/download.pl?mode=txtzip&work=jaochnej	https://www.gutenberg.org/files/18043/18043-0.txt    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
jaochnej_02	https://www.archive.org/download/ja_och_nej_0903_librivox/jaochnej_02_nyblom.mp3	http://runeberg.org/download.pl?mode=txtzip&work=jaochnej	https://www.gutenberg.org/files/18043/18043-0.txt    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
jaochnej_03	https://www.archive.org/download/ja_och_nej_0903_librivox/jaochnej_03_nyblom.mp3	http://runeberg.org/download.pl?mode=txtzip&work=jaochnej	https://www.gutenberg.org/files/18043/18043-0.txt    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
jaochnej_04	https://www.archive.org/download/ja_och_nej_0903_librivox/jaochnej_04_nyblom.mp3	http://runeberg.org/download.pl?mode=txtzip&work=jaochnej	https://www.gutenberg.org/files/18043/18043-0.txt    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
jaochnej_05	https://www.archive.org/download/ja_och_nej_0903_librivox/jaochnej_05_nyblom.mp3	http://runeberg.org/download.pl?mode=txtzip&work=jaochnej	https://www.gutenberg.org/files/18043/18043-0.txt    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
jaochnej_06	https://www.archive.org/download/ja_och_nej_0903_librivox/jaochnej_06_nyblom.mp3	http://runeberg.org/download.pl?mode=txtzip&work=jaochnej	https://www.gutenberg.org/files/18043/18043-0.txt    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
jaochnej_07	https://www.archive.org/download/ja_och_nej_0903_librivox/jaochnej_07_nyblom.mp3	http://runeberg.org/download.pl?mode=txtzip&work=jaochnej	https://www.gutenberg.org/files/18043/18043-0.txt    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
jaochnej_08	https://www.archive.org/download/ja_och_nej_0903_librivox/jaochnej_08_nyblom.mp3	http://runeberg.org/download.pl?mode=txtzip&work=jaochnej	https://www.gutenberg.org/files/18043/18043-0.txt    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_01	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_01_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_02	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_02_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_03	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_03_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_04	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_04_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_05	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_05_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_06	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_06_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_07	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_07_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_08	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_08_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_09	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_09_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_10	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_10_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_11	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_11_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_12	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_12_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna2_13	https://www.archive.org/download/nyckfull_kvinna_del2_lr_librivox/ennyckfullkvinna2_13_flygare-carlen.mp3	https://books.google.com/books?id=whQNAAAAYAAJ    Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
jenny	https://www.archive.org/download/multilingual_1204_librivox/swedish_jenny_runeberg_pa.mp3	https://www.gutenberg.org/cache/epub/13100/pg13100.txt	Petra	https://librivox.org/reader/2909	f
jordensinre_01	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_01_witt.mp3	http://runeberg.org/jordinre/k1.html		Petra	https://librivox.org/reader/2909	f
jordensinre_02	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_02_witt.mp3	http://runeberg.org/jordinre/k2.html		Petra	https://librivox.org/reader/2909	f
jordensinre_03	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_03_witt.mp3	http://runeberg.org/jordinre/k3.html		Petra	https://librivox.org/reader/2909	f
jordensinre_04	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_04_witt.mp3	http://runeberg.org/jordinre/k4.html		Petra	https://librivox.org/reader/2909	f
jordensinre_05	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_05_witt.mp3	http://runeberg.org/jordinre/k5.html		Petra	https://librivox.org/reader/2909	f
jordensinre_06	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_06_witt.mp3	http://runeberg.org/jordinre/k6.html		Petra	https://librivox.org/reader/2909	f
jordensinre_07	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_07_witt.mp3	http://runeberg.org/jordinre/k7.html		Petra	https://librivox.org/reader/2909	f
jordensinre_08	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_08_witt.mp3	http://runeberg.org/jordinre/k8.html		Petra	https://librivox.org/reader/2909	f
jordensinre_09	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_09_witt.mp3	http://runeberg.org/jordinre/k9.html		Petra	https://librivox.org/reader/2909	f
jordensinre_10	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_10_witt.mp3	http://runeberg.org/jordinre/k10.html		Petra	https://librivox.org/reader/2909	f
jordensinre_11	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_11_witt.mp3	http://runeberg.org/jordinre/k11.html		Petra	https://librivox.org/reader/2909	f
jordensinre_12	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_12_witt.mp3	http://runeberg.org/jordinre/k12.html		Petra	https://librivox.org/reader/2909	f
jordensinre_13	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_13_witt.mp3	http://runeberg.org/jordinre/k13.html		Petra	https://librivox.org/reader/2909	f
jordensinre_14	https://www.archive.org/download/jordens_inre_p_librivox/jordensinre_14_witt.mp3	http://runeberg.org/jordinre/k14.html		Petra	https://librivox.org/reader/2909	f
landshovdingen	https://www.archive.org/download/multilingual_poetry_017_1103/swedish_landshovdingen_runeberg_pirv.mp3	http://runeberg.org/fstal/2p.html	Pentti Hirvonen	https://librivox.org/reader/5979	m
nocturne	https://www.archive.org/download/multilingual_poetry_017_1103/swedish_nocturne_sodergran_jb.mp3	http://web.archive.org/web/20180815020322/http://www.svensklyrik.se/poeter/edith-sodergran/nocturne-2	Johan Borg	https://librivox.org/reader/5958	m
sockerpullor_01	https://www.archive.org/download/sockerpullor_pepparkorn_petra_librivox/sockerpullor_01_wraner.mp3	https://www.gutenberg.org/ebooks/31533.txt.utf-8	Petra  https://librivox.org/reader/2909  f
sockerpullor_02	https://www.archive.org/download/sockerpullor_pepparkorn_petra_librivox/sockerpullor_02_wraner.mp3	https://www.gutenberg.org/ebooks/31533.txt.utf-8	Petra  https://librivox.org/reader/2909  f
sockerpullor_03	https://www.archive.org/download/sockerpullor_pepparkorn_petra_librivox/sockerpullor_03_wraner.mp3	https://www.gutenberg.org/ebooks/31533.txt.utf-8	Petra  https://librivox.org/reader/2909  f
sockerpullor_04	https://www.archive.org/download/sockerpullor_pepparkorn_petra_librivox/sockerpullor_04_wraner.mp3	https://www.gutenberg.org/ebooks/31533.txt.utf-8	Petra  https://librivox.org/reader/2909  f
sockerpullor_05	https://www.archive.org/download/sockerpullor_pepparkorn_petra_librivox/sockerpullor_05_wraner.mp3	https://www.gutenberg.org/ebooks/31533.txt.utf-8	Petra  https://librivox.org/reader/2909  f
sockerpullor_06	https://www.archive.org/download/sockerpullor_pepparkorn_petra_librivox/sockerpullor_06_wraner.mp3	https://www.gutenberg.org/ebooks/31533.txt.utf-8	Petra  https://librivox.org/reader/2909  f
sockerpullor_07	https://www.archive.org/download/sockerpullor_pepparkorn_petra_librivox/sockerpullor_07_wraner.mp3	https://www.gutenberg.org/ebooks/31533.txt.utf-8	Petra  https://librivox.org/reader/2909  f
tomten	https://www.archive.org/download/christmasshortworks_2014_1412_librivox/csw14_08_tomten_128kb.mp3	https://www.gutenberg.org/cache/epub/15753/pg15753.txt	J Korth	https://librivox.org/reader/9337	m
ennyckfullkvinna_01	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_01_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Cecilia	https://librivox.org/reader/919	f
ennyckfullkvinna_02	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_02_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Cecilia	https://librivox.org/reader/919	f
ennyckfullkvinna_03	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_03_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Cecilia	https://librivox.org/reader/919	f
ennyckfullkvinna_04	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_04_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Cecilia	https://librivox.org/reader/919	f
ennyckfullkvinna_05	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_05_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Cecilia	https://librivox.org/reader/919	f
ennyckfullkvinna_06	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_06_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Cecilia	https://librivox.org/reader/919	f
ennyckfullkvinna_07	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_07_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna_07	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_07_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna_08	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_08_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna_09	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_09_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna_10	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_10_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna_11	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_11_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna_12	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_12_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna_13	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_13_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna_14	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_14_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna_15	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_15_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna_16	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_16_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna_17	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_17_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
ennyckfullkvinna_18	https://www.archive.org/download/nyckfull_kvinna_del1_1111_librivox/ennyckfullkvinna_18_flygare-carlen.mp3	http://runeberg.org/fcenyck/1/	Lars Rolander (1942-2016) https://librivox.org/reader/2234  m
"""

In [3]:
items = {}
for line in DATA.split("\n"):
    if line.strip() == "":
        continue
    if line.startswith("#"):
        continue
    parts = line.split("\t")
    if f"{parts[0]}.wav" not in DONE:
        items[parts[0]] = parts[1:]

In [ ]:
for item in items.keys():
    !wget {items[item][0]}
    mp3name = items[item][0].split("/")[-1]
    !ffmpeg -i {mp3name} -acodec pcm_s16le -ac 1 -ar 16000 {item}.wav

--2022-07-07 13:32:46--  https://www.archive.org/download/mshortworks_001_1202_librivox/msw001_19_eftertorsdagen_benedictsson_at.mp3
Resolving www.archive.org (www.archive.org)... 207.241.224.2
Connecting to www.archive.org (www.archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://archive.org/download/mshortworks_001_1202_librivox/msw001_19_eftertorsdagen_benedictsson_at.mp3 [following]
--2022-07-07 13:32:47--  https://archive.org/download/mshortworks_001_1202_librivox/msw001_19_eftertorsdagen_benedictsson_at.mp3
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia800801.us.archive.org/0/items/mshortworks_001_1202_librivox/msw001_19_eftertorsdagen_benedictsson_at.mp3 [following]
--2022-07-07 13:32:47--  https://ia800801.us.archive.org/0/items/mshortworks_001_1202_librivox/

Here starts the actual ASR stuff.

In [ ]:
!pip install transformers

In [ ]:
_SWE_MODEL = "KBLab/wav2vec2-large-voxrex-swedish"

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(model=_SWE_MODEL)

In [ ]:
import glob
import json

for file in glob.glob("*.wav"):
    print(file)
    if file in DONE:
        continue
    jsonf = file.replace(".wav", ".json")
    output = pipe(file, chunk_length_s=10, return_timestamps="word")
    with open(jsonf, "w") as f:
        json.dump(output, f)
    print("Wrote", jsonf)